In [1]:
import h5py
import numpy as np

In [2]:
f = h5py.File('data/assorted_images/satellite_images.h5', 'r')
print(np.array(f))

['data' 'labels']


In [3]:
raw_data = np.array(f['data'])
labels = np.array(f['labels'])
print(labels)

[0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0
 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1
 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 

In [4]:
print(raw_data.shape)


(641, 224, 224, 3)


In [5]:
train_ratio, val_ratio = 0.9, 0.05
num = raw_data.shape[0]

# train_num = int(train_ratio*num)
# val_num = int((num-train_num)/2)
# test_num = num-train_num-val_num

# train_data = raw_data[train_num:]
# train_labels = labels[train_num:]
# test_data = raw_data[train_num:(train_num+test_num)]
# test_labels = labels[train_num:(train_num+test_num)]
# val_data = raw_data[:val_num]
# val_labels = labels[:val_num]

# print(val_data.shape)
# print(train_data.shape)
# print(test_data.shape)
shuffled_ind = np.arange(raw_data.shape[0])
np.random.shuffle(shuffled_ind)
shuffled_dset = raw_data[shuffled_ind]
shuffled_lset = labels[shuffled_ind]

train_data = shuffled_dset[:int(raw_data.shape[0]*train_ratio), ...]
train_labels = shuffled_lset[:int(raw_data.shape[0]*train_ratio), ...]

val_data = shuffled_dset[int(raw_data.shape[0]*train_ratio):int(raw_data.shape[0]*(train_ratio+val_ratio)), ...]
val_labels = shuffled_lset[int(raw_data.shape[0]*train_ratio):int(raw_data.shape[0]*(train_ratio+val_ratio)), ...]

test_data = shuffled_dset[int(raw_data.shape[0]*(train_ratio+val_ratio)):, ...]
test_labels = shuffled_lset[int(raw_data.shape[0]*(train_ratio+val_ratio)):, ...]

print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(576, 224, 224, 3)
(33, 224, 224, 3)
(32, 224, 224, 3)


In [6]:
import keras
from keras.models import Sequential # Sequential is one of the main models in Keras, which is basically a sequentially stacked series of layers

model = Sequential() # Initialize a Sequential model instance

Using TensorFlow backend.
W0710 16:08:52.978844 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [7]:
# First we'll use fully-connected neural nets
from keras.layers import Dense # Dense is Keras's name for fully connected layers

# We can stack layers like lego blocks by simplying using `add()`
# `units` is the number of neurons
# `activation` is the nonlinear function we add for each layer
# We only need to specify `input_dim` which is the input dimension for the layer for the input layer, because for later layers the input is just the output from last layer
# Once again, the number of neurons in hidden layers (e.g., 64 and 16 here) are design choices

model.add(Dense(units=64, activation='sigmoid', input_dim=224*224*3)) 
model.add(keras.layers.Dropout(0.5, noise_shape=None, seed=None))
model.add(Dense(units=16, activation='sigmoid'))
model.add(keras.layers.Dropout(0.5, noise_shape=None, seed=None))
model.add(Dense(units=1, activation='sigmoid'))

W0710 16:08:53.034721 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 16:08:53.051466 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 16:08:53.096162 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0710 16:08:53.117784 140737168323520 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Ins

In [8]:
# Once the model is build, we then configure the learning process with `compile()`
# We need to specify the loss function, the optimizer and the metric we use to evaluate our model
# For loss here we're using a function called binary cross-entropy loss, which is specifically for binary classification
# For optimizer we're using gradient descent, which is written as 'sgd' in Keras
# Since we're doing classification, normally the classification accuracy is how we evaluate the model

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

W0710 16:08:53.224805 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0710 16:08:53.262505 140737168323520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0710 16:08:53.273715 140737168323520 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
# Up to this point we're all doing configurations. Now everything is set up so we're letting the model do real things!

# Since now we're using a fully-connected nets, remember we need to flatten the image to a single long vector first
train_data = train_data.reshape((-1, 224*224*3)) # -1 means letting NumPy to figure this axis out automatically
test_data = test_data.reshape((-1, 224*224*3))
val_data = val_data.reshape((-1, 224*224*3))

print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

# Then use fit() to actually train our model
# epochs is basically how many iterations we want for the update process. The model needs some time to reach the optimal state!
# batch_size is how many images we use each time to estimate the gradient. Remember that the more we use the more accurate each update will be, but it will also be slower

model.fit(train_data, train_labels, epochs=30, batch_size=32, validation_data=(val_data, val_labels), shuffle=True)
#epochs - how many iterations
#batch size - sgd batch

(576, 150528)
(33, 150528)
(32, 150528)
Train on 576 samples, validate on 32 samples
Epoch 1/30
576/576 [==============================] - 3s 5ms/step - loss: 0.8370 - acc: 0.4115 - val_loss: 0.7243 - val_acc: 0.2188
Epoch 2/30
576/576 [==============================] - 2s 3ms/step - loss: 0.7357 - acc: 0.5000 - val_loss: 0.6199 - val_acc: 0.7812
Epoch 3/30
576/576 [==============================] - 2s 3ms/step - loss: 0.6317 - acc: 0.6319 - val_loss: 0.5785 - val_acc: 0.7812
Epoch 4/30
576/576 [==============================] - 2s 3ms/step - loss: 0.5840 - acc: 0.6858 - val_loss: 0.5552 - val_acc: 0.7812
Epoch 5/30
576/576 [==============================] - 2s 3ms/step - loss: 0.5814 - acc: 0.7205 - val_loss: 0.5420 - val_acc: 0.7812
Epoch 6/30
576/576 [==============================] - 2s 3ms/step - loss: 0.5793 - acc: 0.7326 - val_loss: 0.5356 - val_acc: 0.7812
Epoch 7/30
576/576 [==============================] - 2s 3ms/step - loss: 0.5695 - acc: 0.7552 - val_loss: 0.5315 - val_acc

In [16]:
# Now let's see how our model does
acc = model.evaluate(test_data, test_labels)
print('The test accuracy is: {}'.format(acc))

# And make predictions
prob = model.predict(test_data) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

print('The predicted probabilities are: {}'.format(prob))
print('The predicted class labels are: {}'.format(label))

33/33 [==============================] - 0s 1ms/step
The test accuracy is: [0.47713189414053253, 0.8181818181818182]
The predicted probabilities are: [[0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289304]
 [0.21289305]]
The predicted class labels are: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
